# System overview
This notebook is an "executable" version of the README file. It demonstrates the concepts presented in Section 4 of the manuscript.

In [ ]:
from deuterium import Variable, to_vec, random_symbols
import symengine as se
import numpy as np

## Symbolic autodiff

In [ ]:
a, b = se.symbols("a, b")
a = Variable(a)
b = Variable(b)
f = lambda a, b: a ** 2 + 2 * np.log(a * np.sqrt(a * b ** 3)) - np.exp(a + 2 * b) + 5
result = f(a, b)
print(result)
result.backward()
print(a.grad)
print(b.grad)

## Numeric autodiff

In [ ]:
a, b = 2.0, 3.0
a = Variable(a)
b = Variable(b)
f = (
    lambda a, b: a ** 2
    + 2 * np.log(a * np.sqrt(a * b ** 3))
    - np.exp(a / 30.0 + 2 * np.sqrt(b))
    + 5
)
result = f(a, b)
print(result)
result.backward()
print(a.grad)
print(b.grad)

## Array computations

In [ ]:
symbols = random_symbols(n=10, prefix="x")  # generate symbols (x_0,...,x_9)
symbol_array = np.array(symbols)
inputs = to_vec(
    symbol_array
)  # converts the numpy array to an array of deuterium variables

kernel = to_vec(
    np.array(random_symbols(3, "k"))
)  # create a convolution kernel and conver to a variable

output = np.convolve(inputs, kernel) # convolve the inputs with the kernel
output.mean().backward() # gradients can be calculated only w.r.t. a scalar, hence the mean operation

print(inputs.flatten()[0].grad.simplify(rational=True))
print(kernel.flatten()[0].grad.simplify(rational=True))

## JIT compilation - full substitution

In [ ]:
symbols = random_symbols(n=10, prefix="x")  # generate symbols (x_0,...,x_9)
symbol_array = np.array(symbols)
inputs = to_vec(
    symbol_array
)  # converts the numpy array to an array of deuterium variables

kernel = to_vec(
    np.array(random_symbols(3, "k"))
)  # create a convolution kernel and conver to a variable

output = np.convolve(inputs, kernel)
output.mean().backward()

print(inputs.flatten()[0].grad.subs({"k_0": 0.5, "k_1": 0.25, "k_2": 0.75, "k_3": 0.8})) #substitute symbols with numeric values into the gradient.

## JIT compilation - partial substitution

In [ ]:
symbols = random_symbols(n=10, prefix="x")  # generate symbols (x_0,...,x_9)
symbol_array = np.array(symbols)
inputs = to_vec(
    symbol_array
)  # converts the numpy array to an array of deuterium variables

kernel = to_vec(
    np.array(random_symbols(3, "k"))
)  # create a convolution kernel and conver to a variable

output = np.convolve(inputs, kernel)
output.mean().backward()

partial_grad = (
    inputs.flatten()[0].grad.subs({"k_0": 0.5, "k_3": 0.8}).simplify(rational=True)
)
print(partial_grad)
full_grad = partial_grad.subs({"k_1": 0.5, "k_2": 0.45}) #substitute the rest
print(full_grad)

## AOT compilation

In [ ]:
symbols = random_symbols(n=10, prefix="x")  # generate symbols (x_0,...,x_9)
symbol_array = np.array(symbols)
inputs = to_vec(
    symbol_array
)  # converts the numpy array to an array of deuterium variables

kernel = to_vec(
    np.array(random_symbols(3, "k"))
)  # create a convolution kernel and conver to a variable

output = np.convolve(inputs, kernel)
output.mean().backward()

grad = [i.grad for i in kernel.flatten()] # gradients w.r.t kernel

all_symbols = symbols + [s.data for s in kernel.flatten().tolist()] #variables in the function signature

gradient_func = se.Lambdify(all_symbols, grad, cse=True, backend="llvm") #compile with CSE and the LLVM back-end

values = np.random.random((4, len(all_symbols))) #random "batch" of inputs with size 4.

print(gradient_func(values)) #will print the gradient for every entry in the "batch"

## Persistence to `pickle`

In [ ]:
import pickle
_ = pickle.dumps(gradient_func)